In [1]:
import warnings
warnings.filterwarnings("ignore")
from shapely import wkt
import glob


%load_ext autoreload
%autoreload 2
%env PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj
%matplotlib inline

import skimage

from tqdm import tqdm
import numpy as np
import cv2
from functools import partial

from fastai.imports import *
from fastai.vision import *
from fastai.metrics import dice
from fastai.callbacks import *

from joblib import Parallel, delayed
import torch.nn.functional as F
import torch
import functools, traceback
def gpu_mem_restore(func):
    """Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

os.environ['FASTAI_TB_CLEAR_FRAMES']="1"

from catalyst.contrib.criterion import FocalLossMultiClass, DiceLoss, BCEDiceLoss, LovaszLossBinary,LovaszLossMultiClass

from importlib import reload

env: PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj


In [2]:
data_dir = Path("/media/wwymak/Storage/xView2")
mask_dir = data_dir /"mask_full_size"
models_path = data_dir / "models"
train_images_crops = data_dir/"train_crops"
mask_dir_edt = data_dir /"mask_full_size_edt"
train_mask_crops = data_dir/"mask_crops_single_channel"
label_dir = data_dir/"train"/"labels"

In [3]:
label_dir.ls()[0]

PosixPath('/media/wwymak/Storage/xView2/train/labels/socal-fire_00000521_post_disaster.json')

In [4]:
def parse_json(label_json):
    df_xy = pd.DataFrame([{'id': x['properties']['uid'], 'feature': x['properties']['feature_type'],
                           'damage': x['properties'].get('subtype', 'no-damage'),
                           'disaster_type': x['metadata']['disaster_type'],
                           'geometry_pixel': x['wkt']} for x in label_json['features']['xy']])
    df_lnglat = pd.DataFrame(
        [{'id': x['properties']['uid'], 'geometry_lnglat': x['wkt'], } for x in label_json['features']['lng_lat']])
    if len(df_xy) == 0:
        return
    df_xy.set_index('id', inplace=True)

    df_xy['damage_cls'] = df_xy['damage'].map({
        'no-damage': 1, 'minor-damage': 2, 'major-damage': 3, 'destroyed': 4, 'un-classified': 5
    })
    df_xy.geometry_pixel = df_xy.geometry_pixel.apply(wkt.loads)
    df_lnglat.set_index('id', inplace=True)
    label_df = df_xy.merge(df_lnglat['geometry_lnglat'], left_index=True, right_index=True)

    label_gdf = gpd.GeoDataFrame(label_df, geometry='geometry_pixel')
    label_gdf['centroid'] = label_gdf.geometry_pixel.centroid
    return label_gdf

# def create_mask(json_file, data_dir, mask_dir, tile_size=1024):
#     with open(data_dir/"train"/"labels"/json_file, 'r') as f:
#         label = json.load(f)
#     label_gdf = parse_json(label)
#     if label_gdf is None or len(label_gdf) == 0:
#         return 0
#     fb_mask = sol.vector.mask.footprint_mask(df=label_gdf, geom_col="geometry_pixel",
#                                          burn_value=None, burn_field="damage_cls",
#                                          shape=(tile_size,tile_size)).squeeze()
#     # print(fb_mask.max)
#     PIL.Image.fromarray(fb_mask, mode='L').save(mask_dir/(json_file.replace('.json', '.png')))
#     return fb_mask.max()

In [12]:
def disaster_name_to_type(disaster_name):
    if 'volcano' in disaster_name:
        return 'volcano'
    if 'hurricane' in disaster_name:
        return 'hurricane'
    if 'earthquake' in disaster_name:
        return 'earthquake'
    if 'flooding' in disaster_name:
        return 'flooding'
    if 'tsunami' in disaster_name:
        return 'tsunami'
    if 'fire' in disaster_name:
        return 'fire'
    retirn 'unknown'

In [13]:
df = pd.read_csv(data_dir/"train_pre_post.csv")
df['disaster_type'] = df.img_id_pre.apply(lambda x: disaster_name_to_type(x.split("_")[0]))
list(sorted(df.disaster_type.unique()))

['earthquake', 'fire', 'flooding', 'hurricane', 'tsunami', 'volcano']